- Evaluación de contenido 2 Modulo 2

1. Utilizando la API extraed toda la información que podáis de ella. La url para hacer las llamadas es:

In [1]:
from IPython.core.interactiveshell import InteractiveShell 
InteractiveShell.ast_node_interactivity = "all" 

import requests
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from geopy.geocoders import Nominatim
import mysql.connector

In [2]:

paises = ("Argentina", "Canada", "United States")
datos_paises={}
for pais in  paises:
    url = f"http://universities.hipolabs.com/search?country={pais}"
    response=requests.get(url=url)
    datos_universidades=pd.DataFrame(response.json())
    datos_paises[pais]=datos_universidades
    
for pais, datos in datos_paises.items():
 
    argentina_df = datos_paises["Argentina"]
    canada_df = datos_paises["Canada"]
    usa_df = datos_paises["United States"]
   
df_unis = pd.concat([argentina_df,canada_df , usa_df],axis =0)
df_unis  
   

,state-province,domains,name,country,web_pages,alpha_two_code
0,Buenos Aires,[atlantida.edu.ar],Universidad Atlantida Argentina,Argentina,[http://www.atlantida.edu.ar/],AR
1,Buenos Aires,[austral.edu.ar],Universidad Austral Buenos Aires,Argentina,[http://www.austral.edu.ar/],AR
2,Ciudad Autónoma de Buenos Aires,[caece.edu.ar],"Universidad CAECE, Buenos Aires",Argentina,[http://www.caece.edu.ar/],AR
3,Ciudad Autónoma de Buenos Aires,[cema.edu.ar],Instituto Universitario CEMA,Argentina,[http://www.cema.edu.ar/],AR
4,Ciudad Autónoma de Buenos Aires,[iese.edu.ar],Instituto de Enseñanza Superior del Ejército,Argentina,[http://www.iese.edu.ar/],AR
...,...,...,...,...,...,...
2276,None,[vul.edu],Virginia University of Lynchburg,United States,[https://www.vul.edu/],US
2277,None,[voorhees.edu],Voorhees University,United States,[https://www.voorhees.edu/],US
2278,None,[wvstate.edu],West Virginia State University,United States,[https://www.wvstateu.edu/],US
2279,None,[wileyc.edu],Wiley College,United States,[https://www.wileyc.edu/],US


2. Una vez tengáis todos los datos de la API, deberéis realizar una serie de procesos de limpieza, estos incluyen:

- Cambiad los nombres de las columnas para homogeneizarlas, tenemos columnas que tienen - y otras _. Unifícalo para que todo vaya con _.

In [3]:
nuevas_columnas = {col: col.replace("-", "_").lower() for col in df_unis.columns}
df_unis.rename(columns = nuevas_columnas, inplace = True)
df_unis.head(5)

,state_province,domains,name,country,web_pages,alpha_two_code
0,Buenos Aires,[atlantida.edu.ar],Universidad Atlantida Argentina,Argentina,[http://www.atlantida.edu.ar/],AR
1,Buenos Aires,[austral.edu.ar],Universidad Austral Buenos Aires,Argentina,[http://www.austral.edu.ar/],AR
2,Ciudad Autónoma de Buenos Aires,[caece.edu.ar],"Universidad CAECE, Buenos Aires",Argentina,[http://www.caece.edu.ar/],AR
3,Ciudad Autónoma de Buenos Aires,[cema.edu.ar],Instituto Universitario CEMA,Argentina,[http://www.cema.edu.ar/],AR
4,Ciudad Autónoma de Buenos Aires,[iese.edu.ar],Instituto de Enseñanza Superior del Ejército,Argentina,[http://www.iese.edu.ar/],AR


- La columna de domains nos da una información similar a la de web_pages. Eliminad la columna domains.

In [4]:
df_unis.drop(["domains"], axis = 1, inplace = True)


In [5]:
df_unis.head(3)

,state_province,name,country,web_pages,alpha_two_code
0,Buenos Aires,Universidad Atlantida Argentina,Argentina,[http://www.atlantida.edu.ar/],AR
1,Buenos Aires,Universidad Austral Buenos Aires,Argentina,[http://www.austral.edu.ar/],AR
2,Ciudad Autónoma de Buenos Aires,"Universidad CAECE, Buenos Aires",Argentina,[http://www.caece.edu.ar/],AR


In [6]:
df_unis.tail()

,state_province,name,country,web_pages,alpha_two_code
2276,None,Virginia University of Lynchburg,United States,[https://www.vul.edu/],US
2277,None,Voorhees University,United States,[https://www.voorhees.edu/],US
2278,None,West Virginia State University,United States,[https://www.wvstateu.edu/],US
2279,None,Wiley College,United States,[https://www.wileyc.edu/],US
2280,None,Winston-Salem State University,United States,[https://www.wssu.edu/],US


3. Si exploramos la columna de web_pages, nos daremos cuenta que hay universidades, como por ejemplo la Universidad de "Cégep de Saint-Jérôme" de Canadá que en su columna de web_pages tiene más de un valor dentro de la lista. Esto es poco práctico y puede llegar a no tener sentido. el objetivo de este ejericio es que usando el método explode de pandas separéis cada elemento de la lista en una fila nueva. 

In [7]:
df_unis = df_unis.explode("web_pages")


In [8]:
df_unis.sample(3)

,state_province,name,country,web_pages,alpha_two_code
120,Quebec,Université du Québec en Outaouais,Canada,http://www.uqo.ca/,CA
1007,None,University of Illinois at Chicago,United States,http://www.uic.edu/,US
1127,None,"University of Virginia, Charlottesville",United States,http://www.virginia.edu/,US


4. Una vez hayáis realizado el explode, chequead si tenéis duplicados basándonos unicamente en el nombre de la universidad, en caso de que si, eliminandlos.

In [9]:
df_unis.duplicated("name").sum()

28

In [10]:
df_unis[df_unis.duplicated(["name"])].head(2)

,state_province,name,country,web_pages,alpha_two_code
0,Quebec,Cégep de Saint-Jérôme,Canada,https://ccmt.cstj.qc.ca,CA
0,Quebec,Cégep de Saint-Jérôme,Canada,https://ccml.cstj.qc.ca,CA


In [11]:
df_unis.drop_duplicates(subset=["name"], inplace = True)


In [12]:
df_unis.duplicated("name").sum()

0

5. Si exploramos la columna de state_province veremos que hay universidades cuyo valor para esta columna es None. Cread una función para reemplazar los None por nulos de numpy.

In [13]:
df_unis["state_province"]

0                          Buenos Aires
1                          Buenos Aires
2       Ciudad Autónoma de Buenos Aires
3       Ciudad Autónoma de Buenos Aires
4       Ciudad Autónoma de Buenos Aires
                     ...               
2276                               None
2277                               None
2278                               None
2279                               None
2280                               None
Name: state_province, Length: 2507, dtype: object

In [14]:
df_unis["state_province"] = df_unis["state_province"].replace({None:np.nan})

In [15]:
df_unis["state_province"]

0                          Buenos Aires
1                          Buenos Aires
2       Ciudad Autónoma de Buenos Aires
3       Ciudad Autónoma de Buenos Aires
4       Ciudad Autónoma de Buenos Aires
                     ...               
2276                                NaN
2277                                NaN
2278                                NaN
2279                                NaN
2280                                NaN
Name: state_province, Length: 2507, dtype: object

6. Después del último cambio, os habréis dado cuenta que tenemos muchos valores nulos dentro de la columna de state_province, por lo que nuestro jefe nos pide que reemplacemos esos nulos por "Unknow". No nos piden ningún método especifico, asi que podremos usar el método que queramos.

In [16]:
df_unis["state_province"].replace(np.nan, "Unknow", inplace = True)

In [17]:
df_unis.isnull().sum()

state_province    0
name              0
country           0
web_pages         0
alpha_two_code    0
dtype: int64

7. Ahora nuestros jefes nos piden que saquemos las coordenadas de las provincias donde están ubicadas las universidades. Para eso nos piden que usemos la librería de geopy que aprendimos el día del repaso,
-  Sacar los valores únicos de la columna state_province.

In [18]:
df_unis["state_province"].unique()

array(['Buenos Aires', 'Ciudad Autónoma de Buenos Aires', 'Entre Ríos',
       'Salta', 'Córdoba', 'Mendoza', 'Santa Fé', 'Unknow',
       'Santiago Del Estero', 'Misiones', 'Catamarca', 'Formosa', 'Jujuy',
       'La Rioja', 'La Pampa', 'San Juan', 'San Luis', 'Tucumán',
       'Quebec', 'Ontario', 'Nova Scotia', 'British Columbia', 'Alberta',
       'Manitoba', 'New Brunswick', 'Saskatchewan',
       'Newfoundland and Labrador', 'Prince Edward Island', 'Yukon',
       'Pennsylvania', 'NV', 'Iowa', 'VA', 'TX', 'Colorado', 'IN', 'CA',
       'South Carolina', 'Washington', 'NY', 'Texas', 'ND', 'MI', 'Ohio',
       'Florida', 'California', 'North Carolina', 'Michigan', 'GA',
       'New York, NY'], dtype=object)

- Algunos de los valores que tenemos están con siglas, y deberéis reemplazarlos por lo siguiente:

NV: reemplazalo por Nevada
TX: reemplazalo por Texas
IN: reemplazalo por Indianapolis
CA: reemplazalo por California
VA: reemplazalo por Virginia
NY: reemplazalo por New York
MI: reemplazalo por Michigan
GA: reemplazalo por Georgia
ND: reemplazalo por North Dakota

In [19]:
remplazo_nombre={"NV":  " Nevada",
                "TX":  "Texas",
                "IN":  "Indianapolis",
                "CA":  "California",
                "VA":"Virginia",
                "NY": " New York",
                "MI":  "Michigan",
                "GA":  "Georgia",
                "ND":  "North Dakota"}

In [20]:
df_unis["state_province"].replace(remplazo_nombre, inplace=True)

In [21]:
df_unis["state_province"].unique()

array(['Buenos Aires', 'Ciudad Autónoma de Buenos Aires', 'Entre Ríos',
       'Salta', 'Córdoba', 'Mendoza', 'Santa Fé', 'Unknow',
       'Santiago Del Estero', 'Misiones', 'Catamarca', 'Formosa', 'Jujuy',
       'La Rioja', 'La Pampa', 'San Juan', 'San Luis', 'Tucumán',
       'Quebec', 'Ontario', 'Nova Scotia', 'British Columbia', 'Alberta',
       'Manitoba', 'New Brunswick', 'Saskatchewan',
       'Newfoundland and Labrador', 'Prince Edward Island', 'Yukon',
       'Pennsylvania', ' Nevada', 'Iowa', 'Virginia', 'Texas', 'Colorado',
       'Indianapolis', 'California', 'South Carolina', 'Washington',
       ' New York', 'North Dakota', 'Michigan', 'Ohio', 'Florida',
       'North Carolina', 'Georgia', 'New York, NY'], dtype=object)

- Otros valores que tenemos más formateados son y que deberemos reemplazar:
- New York, NY. Deberéis reemplazarlo por "New York".
- 'Buenos Aires', 'Ciudad Autónoma de Buenos Aires'. En este caso deberéis poner en ambos casos "Buenos Aires"

In [22]:
remplazo_nuevo={'New York, NY':"New York",
                "Buenos Aires": "Buenos Aires",
                'Ciudad Autónoma de Buenos Aires': 'Buenos Aires'}

In [23]:
df_unis["state_province"].replace(remplazo_nuevo, inplace=True)

In [24]:
df_unis["state_province"].unique()

array(['Buenos Aires', 'Entre Ríos', 'Salta', 'Córdoba', 'Mendoza',
       'Santa Fé', 'Unknow', 'Santiago Del Estero', 'Misiones',
       'Catamarca', 'Formosa', 'Jujuy', 'La Rioja', 'La Pampa',
       'San Juan', 'San Luis', 'Tucumán', 'Quebec', 'Ontario',
       'Nova Scotia', 'British Columbia', 'Alberta', 'Manitoba',
       'New Brunswick', 'Saskatchewan', 'Newfoundland and Labrador',
       'Prince Edward Island', 'Yukon', 'Pennsylvania', ' Nevada', 'Iowa',
       'Virginia', 'Texas', 'Colorado', 'Indianapolis', 'California',
       'South Carolina', 'Washington', ' New York', 'North Dakota',
       'Michigan', 'Ohio', 'Florida', 'North Carolina', 'Georgia',
       'New York'], dtype=object)

- Una vez realizados los pasos anteriores, crea una lista con los valores únicos de las provincias de las universidades.

In [25]:
ubicaciones=list(df_unis["state_province"].unique())
ubicaciones

['Buenos Aires',
 'Entre Ríos',
 'Salta',
 'Córdoba',
 'Mendoza',
 'Santa Fé',
 'Unknow',
 'Santiago Del Estero',
 'Misiones',
 'Catamarca',
 'Formosa',
 'Jujuy',
 'La Rioja',
 'La Pampa',
 'San Juan',
 'San Luis',
 'Tucumán',
 'Quebec',
 'Ontario',
 'Nova Scotia',
 'British Columbia',
 'Alberta',
 'Manitoba',
 'New Brunswick',
 'Saskatchewan',
 'Newfoundland and Labrador',
 'Prince Edward Island',
 'Yukon',
 'Pennsylvania',
 ' Nevada',
 'Iowa',
 'Virginia',
 'Texas',
 'Colorado',
 'Indianapolis',
 'California',
 'South Carolina',
 'Washington',
 ' New York',
 'North Dakota',
 'Michigan',
 'Ohio',
 'Florida',
 'North Carolina',
 'Georgia',
 'New York']

-Usando la API de geopy, extraed la latitud y la longitud de cada una de las provincias y almacenad los resultados en un dataframe.

In [26]:
geolocator=Nominatim(user_agent="Jacque")
location = geolocator.geocode("Buenos Aires")
location[1]

(-34.6075682, -58.4370894)

In [41]:
def busqueda_geopy(lista):
    try:
        geolocator=Nominatim(user_agent="Jacque")
        location= geolocator.geocode(lista)
        return location[1]
        print(location)
    except:
        return np.nan

In [50]:
cordenadas=[]
provincia=[]


for ubicacion in ubicaciones:
    cordenadas.append(busqueda_geopy(ubicacion))
       
    provincia.append(ubicacion)
   
   

In [44]:
df=pd.DataFrame(cordenadas, columns =["latitud", "longitud"])
df1=pd.DataFrame(provincia, columns=["state_province"])
df_entero=pd.concat([df,df1],axis=1)

In [45]:
df1.head(2)

,state_province
0,Buenos Aires
1,Entre Ríos


In [46]:
df_entero.head()

,latitud,longitud,state_province
0,-34.607568,-58.437089,Buenos Aires
1,-31.625284,-59.353958,Entre Ríos
2,-25.107670,-64.349496,Salta
3,37.884581,-4.776014,Córdoba
4,-34.787093,-68.438187,Mendoza


- Una vez que tengáis los datos del ejercicio anterior en un dataframe, unidlo con el de las universidades que hemos sacado de la API.

In [47]:
df_definitivo=df_unis.merge(df_entero, on="state_province")
df_definitivo

,state_province,name,country,web_pages,alpha_two_code,latitud,longitud
0,Buenos Aires,Universidad Atlantida Argentina,Argentina,http://www.atlantida.edu.ar/,AR,-34.607568,-58.437089
1,Buenos Aires,Universidad Austral Buenos Aires,Argentina,http://www.austral.edu.ar/,AR,-34.607568,-58.437089
2,Buenos Aires,"Universidad CAECE, Buenos Aires",Argentina,http://www.caece.edu.ar/,AR,-34.607568,-58.437089
3,Buenos Aires,Instituto Universitario CEMA,Argentina,http://www.cema.edu.ar/,AR,-34.607568,-58.437089
4,Buenos Aires,Instituto de Enseñanza Superior del Ejército,Argentina,http://www.iese.edu.ar/,AR,-34.607568,-58.437089
...,...,...,...,...,...,...,...
2502,Georgia,Morehouse,United States,https://morehouse.edu/,US,32.329381,-83.113737
2503,New York,CUNY Lehman College,United States,https://www.lehman.cuny.edu/,US,40.712728,-74.006015
2504,New York,CUNY John Jay College of Criminal Justice,United States,https://www.jjay.cuny.edu/,US,40.712728,-74.006015
2505,New York,CUNY Hunter College,United States,https://hunter.cuny.edu/,US,40.712728,-74.006015


In [48]:
df_definitivo.to_pickle("datos/datos_unis.pickle")

8. Crea una BBDD en mysql que contenga las siguientes tablas:
- idestado: primary key, integer, autoincremental
   - nombre_pais: varchar
   - nombre_provincia: varchar
   - latitud: decimal
   - longitud: decimal
- tabla universidades: donde encontramos las siguientes columnas:
   - iduniversidades: primary key, integer, autoincremental
   - nombre_universidad: varchar
   - pagina_web: varchar
   - paises_idestado: foreing key   



9. Introduce todo el código que habéis ido creando en funciones, siguiendo la misma lógica que hemos seguido en los pairs

1 creación de la clase

1 definicion de la variable

In [33]:

tabla_pais = """

CREATE TABLE IF NOT EXISTS `mydb`.`paises` (
  `idpaisesididestado` INT NOT NULL AUTO_INCREMENT,
  `nombre_pais:` VARCHAR(45) NULL,
  `nombre_provincia` VARCHAR(45) NULL,
  `latitud` DECIMAL NULL,
  `Longitud` DECIMAL NULL,
  PRIMARY KEY (`idpaisesididestado`))
ENGINE = InnoDB;"""

tabla_universidades = """

CREATE TABLE IF NOT EXISTS `mydb`.`universidades` (
  `iduniversidades` INT NOT NULL AUTO_INCREMENT,
  `nombre_universidad` VARCHAR(45) NULL,
  `pagina_web` VARCHAR(45) NULL,
  `paises_idestado` INT NOT NULL,
  PRIMARY KEY (`iduniversidades`),
  INDEX `fk_universidades_paises_idx` (`paises_idestado` ASC) VISIBLE,
  CONSTRAINT `fk_universidades_paises`
    FOREIGN KEY (`paises_idestado`)
    REFERENCES `mydb`.`paises` (`idpaisesididestado`)
    ON DELETE NO ACTION
    ON UPDATE NO ACTION)
ENGINE = InnoDB;"""




In [34]:
class cargarbbdd:
    
    def __init__(self, nombre_bbdd, contraseña):
        
        self.nombre_bbdd = nombre_bbdd
        self.contraseña = contraseña
       

    def creacion_bbdd(self):

        mydb = mysql.connector.connect(host="localhost",
                                        user="root",
                                        password=f'{self.contraseña}')
                                       
        mycursor = mydb.cursor()
        print("conexión realizada")

        try:
            mycursor.execute(f"CREATE DATABASE IF NOT EXISTS {self.nombre_bbdd};")
            print(mycursor)
        except mysql.connector.Error as err:
            print(err)
            print("Error Code:", err.errno)
            print("SQLSTATE", err.sqlstate)
            print("Message", err.msg)



    def crear_insertar_tabla(self,query):
    
        mydb = mysql.connector.connect(user='root', password=f"{self.contraseña}",
                                        host='localhost', database=f"{self.nombre_bbdd}") 
                                        

        mycursor = mydb.cursor()
        
        try: 
            mycursor.execute(query)
            mydb.commit() 

        except mysql.connector.Error as err:
            print(err)
            print("Error Code:", err.errno)
            print("SQLSTATE", err.sqlstate)
            print("Message", err.msg)

1 llamada a la clase

In [35]:
bbdd = cargarbbdd("Universidades", "AlumnaAdalab")

In [36]:
bbdd



In [37]:


bbdd.creacion_bbdd()



conexión realizada
CMySQLCursor: CREATE DATABASE IF NOT EXISTS Universida..


In [38]:


bbdd.creacion_insercion_tabla(tabla_pais)
bbdd.creacion_insercion_tabla(tabla_universidades)





AttributeError: 'cargarbbdd' object has no attribute 'creacion_insercion_tabla'